# Lesson: Grounding in Reality with External Information

Welcome to this lesson on **Grounding**. This is how you connect your AI to sources of real-time, factual information, transforming it from a knowledgeable LLM into an up-to-date expert.

### The Concept
Grounding is the process of connecting the Gemini model to an external information source to ensure its answers are factual and current. There are two primary ways to do this:
1.  **Google Search:** Allows the model to search the web for up-to-the-minute information.
2.  **URL Context:** Forces the model to base its answers on the content of a specific webpage you provide.

In this notebook, we'll explore both methods and even combine them.

In [1]:
#@title 1. Setup
# Install the Google AI Python SDK
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.15.1 requires google-genai!=1.37.0,!=1.38.0,!=1.39.0,<=1.40.0,>=1.21.1, but you have google-genai 1.42.0 which is incompatible.


In [2]:
#@title 2. Configure your API Key
# Use the "Secrets" tab in Colab (click the key icon on the left) to store your
# API key with the name "GOOGLE_API_KEY".

from google import genai
from google.colab import userdata

try:
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
    print('Secret not found. Please add your GOOGLE_API_KEY to the Colab Secrets Manager.')

## Part 1: The Problem - A Model Without Grounding

First, let's see what happens when we ask a standard model a question about a very recent event. Its knowledge will be outdated.

In [10]:
# Ask a time-sensitive question.
from IPython.display import Markdown, display, HTML
prompt = "Who won the Nobel Prize in Physics for the year 2025?"

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt
)

display(Markdown(response.text))

The Nobel Prize in Physics for the year 2025 has not been announced yet.

The Nobel Prizes are typically announced in October of the award year. We will find out the 2025 laureates in October 2025.

## Part 2: Grounding with Google Search

Now, let's give our model the power of Google Search. We do this by passing `config={"tools": [{"google_search": {}}]},` to the model.

In [11]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt,
    config={"tools": [{"google_search": {}}]}
)

display(Markdown(response.text))
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

The Nobel Prize in Physics for the year 2025 has been awarded to John Clarke, Michel H. Devoret, and John M. Martinis. The Royal Swedish Academy of Sciences in Stockholm announced the laureates on October 7, 2025.

These scientists were recognized for their groundbreaking work on macroscopic quantum phenomena in electrical circuits. Their pioneering experiments demonstrated quantum tunneling and energy quantization on a macroscopic scale, significantly advancing the understanding of quantum mechanics in engineered systems. This research is also noted for paving the way for advanced quantum technology, including quantum computing.

Search Query: ['Nobel Prize in Physics 2025 announcement date', 'when are Nobel Prizes announced 2025']
Search Pages: nobelprize.org, vajiramandravi.com, aljazeera.com, britannica.com, time.com


## Part 3: Grounding with a Specific URL

What if you want the model to answer questions based *only* on a specific document or webpage? You can provide a URL as the grounding source. This is perfect for building a "Document Q&A" bot.

The URL Context tool empowers Gemini models to directly access and process content from specific web page URLs you provide within your API requests. This is incredibly interesting because it allows your applications to dynamically interact with live web information without needing you to manually pre-process and feed that content to the model.

URL Context is effective because it allows the models to base its responses and analysis directly on the content of the designated web pages. Instead of relying solely on its general training data or broad web searches (which are also valuable grounding tools), URL Context anchors the model's understanding to the specific information present at those URLs.

Let's create an expert on the James Webb Space Telescope using its Wikipedia page.

In [12]:
prompt = "Based on https://en.wikipedia.org/wiki/James_Webb_Space_Telescope, what are the four key scientific instruments on the James Webb Space Telescope?"
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt,
    config={"tools": [{"url_context": {}}]}
)
display(Markdown(response.text))

The James Webb Space Telescope (JWST) is equipped with four key scientific instruments:
*   **NIRCam** (Near Infrared Camera) is an infrared imager covering a spectral range from 0.6 μm (visible red) to 5 μm (near-infrared). It also functions as the observatory's wavefront sensor, essential for aligning and focusing the main mirror segments.
*   **NIRSpec** (Near Infrared Spectrograph) performs spectroscopy over the same wavelength range as NIRCam. It offers low-resolution, multi-object, and integral field unit/long-slit spectroscopy modes.
*   **MIRI** (Mid-Infrared Instrument) measures the mid-to-long-infrared wavelength range from 5 to 27 μm. It includes both a mid-infrared camera and an imaging spectrometer and must be kept extremely cold (below 6 K) for optimal operation.
*   **FGS/NIRISS** (Fine Guidance Sensor and Near Infrared Imager and Slitless Spectrograph) is used to stabilize the observatory's line-of-sight during scientific observations. The NIRISS module also provides astronomical imaging and spectroscopy in the 0.8 to 5 μm wavelength range. While often referred to together, the FGS is for support and NIRISS is a scientific instrument.

## Part 4: Mixing Grounding Methods (URL + Search)

You can give the model multiple tools at once. Let's create a "Fact-Checker" that can both summarize a provided news article and use Google Search to find more recent information on the topic.

In [13]:
#@title Ask a multi-step question that requires both tools
prompt = """
First, please summarize the main points of the provided NASA article https://www.nasa.gov/news-release/nasa-isro-satellite-lifts-off-to-track-earths-changing-surfaces/.
Then, use Google Search to see if there have been any new updates or announcements
about the NISAR (NASA-ISRO Synthetic Aperture Radar) satellite in the days since that article was published.
"""

# Use both tools
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt,
    config={"tools": [{"url_context": {}}, {"google_search": {}}]}
)
display(Markdown(response.text))

The NASA-ISRO Synthetic Aperture Radar (NISAR) satellite, a joint mission by NASA and the Indian Space Research Organisation (ISRO), successfully launched on July 30, 2025, from India. The satellite, carrying advanced L-band and S-band radar systems, will provide an unprecedented three-dimensional view of Earth's changing surfaces, monitoring land and ice movement, ecosystems, and natural disasters. NISAR will orbit 464 miles (747 kilometers) above Earth, observing nearly all land and ice-covered surfaces every 12 days, even through clouds and darkness, to provide crucial data for disaster response, infrastructure monitoring, and agricultural management. The mission is expected to have a 90-day commissioning phase, during which its 39-foot (12-meter) radar antenna reflector will deploy.

Next, I will search for recent updates on the NISAR satellite.The NASA-ISRO Synthetic Aperture Radar (NISAR) satellite successfully launched on July 30, 2025, through a collaborative effort between NASA and the Indian Space Research Organisation (ISRO). The satellite's mission is to provide detailed, three-dimensional views of Earth's changing surfaces, tracking land and ice movement, monitoring ecosystems, and aiding in natural disaster response and agricultural management. It is equipped with advanced L-band and S-band radar systems, enabling it to "see" through clouds and darkness to observe nearly all land and ice-covered surfaces every 12 days from an orbit of 464 miles (747 kilometers) above Earth.

Since its launch, the NISAR satellite has made significant progress in its commissioning phase. On August 15, 2025, its 39-foot (12-meter) radar antenna reflector was successfully deployed. Following this, engineers powered on both the L-band and S-band synthetic aperture radar systems. By August 26, the mission team commanded the spacecraft to begin ascending into its operational orbit. The satellite also sent back its first radar images of Earth's surface, which were released on September 25, 2025, showcasing its ability to scan Earth with high detail for various applications like disaster response, infrastructure monitoring, and agricultural management. These initial images, captured by the L-band SAR system, which was provided by NASA's Jet Propulsion Laboratory, demonstrate its capacity to resolve objects as small as 15 feet (5 meters). The mission is on track to commence full science operations later in the fall.